In [1]:
import os
import sys

notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_dir = os.path.dirname(notebook_dir)

if project_dir not in sys.path:
    sys.path.append(project_dir)

In [2]:
# Import Data
import pandas as pd
tweets_df = pd.read_csv(r"C:\Users\johna\anaconda3\envs\twitter-influence-env\twitter-influence\data\01_raw\top_accounts_details.csv")
tweets_df['date'] = pd.to_datetime(tweets_df['creation_date'], format='%a %b %d %H:%M:%S +0000 %Y')
tweets_df['month_year'] = tweets_df['date'].apply(lambda x: x.strftime('%Y-%m'))
tweets_df['date_day'] = tweets_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
tweets_df.head()

,tweet_id,creation_date,text,username,name,follower_count,following_count,number_of_tweets,is_verified,is_blue_verified,...,retweet_count,quote_count,reply_count,retweet,video_view_count,in_reply_to_status_id,quoted_status_id,date,month_year,date_day
0,1640814094822383617,Tue Mar 28 20:32:19 +0000 2023,Grateful to all the first responders and volun...,BarackObama,Barack Obama,133043376,562688,16792,True,False,...,970,31,304,False,NaN,NaN,NaN,2023-03-28 20:32:19,2023-03,2023-03-28
1,1640485381459025920,Mon Mar 27 22:46:08 +0000 2023,We are failing our children. Guns are now the ...,BarackObama,Barack Obama,133043376,562688,16792,True,False,...,23262,2182,16822,False,2826094.0,NaN,NaN,2023-03-27 22:46:08,2023-03,2023-03-27
2,1639039698675937281,Thu Mar 23 23:01:30 +0000 2023,I hope their stories inspire you to help us pr...,BarackObama,Barack Obama,133043376,562688,16792,True,False,...,362,27,547,False,NaN,1.639040e+18,NaN,2023-03-23 23:01:30,2023-03,2023-03-23
3,1639039601623851009,Thu Mar 23 23:01:07 +0000 2023,To celebrate the 13th anniversary of the Affor...,BarackObama,Barack Obama,133043376,562688,16792,True,False,...,2045,152,859,False,442808.0,NaN,NaN,2023-03-23 23:01:07,2023-03,2023-03-23
4,1638956751910215711,Thu Mar 23 17:31:54 +0000 2023,I know so many of you have a story of how acce...,BarackObama,Barack Obama,133043376,562688,16792,True,False,...,641,33,318,False,NaN,1.638957e+18,NaN,2023-03-23 17:31:54,2023-03,2023-03-23


In [3]:
from scripts.text_processing.sentiment_analysis import SentimentAnalyzer

# sentiment analyser, specify model
analyzer = SentimentAnalyzer('cardiffnlp/twitter-roberta-base-sentiment-latest')

# Get sentiment analysis
tweets_with_sentiment = analyzer.get_sentiment(tweets_df)


C:\Users\johna\anaconda3\envs\twitter-influence-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
influence_metrics = tweets_with_sentiment.groupby('username').agg(
    {
        'favorite_count': ['sum', 'mean'],
        'retweet_count': ['sum', 'mean'],
        'quote_count': ['sum', 'mean'],
        'reply_count': ['sum', 'mean'],
        'tweet_id': ['count']
    }
)

# Flatten the MultiIndex columns
influence_metrics.columns = ['_'.join(col).strip() for col in influence_metrics.columns.values]

# Reset the index to move the 'username' back to a column
influence_metrics.reset_index(inplace=True)
influence_metrics

,username,favorite_count_sum,favorite_count_mean,retweet_count_sum,retweet_count_mean,quote_count_sum,quote_count_mean,reply_count_sum,reply_count_mean,tweet_id_count
0,24JulyFirework,5558,5558.000000,382,382.000000,119,119.000000,435,435.0,1
1,4rblinder,9763,9763.000000,1532,1532.000000,25,25.000000,180,180.0,1
2,ACLU,7719,3859.500000,2879,1439.500000,84,42.000000,105,52.5,2
3,AFIFEST,5564,5564.000000,1141,1141.000000,39,39.000000,375,375.0,1
4,AGT,1462,1462.000000,190,190.000000,8,8.000000,131,131.0,1
...,...,...,...,...,...,...,...,...,...,...
748,yasmeenhussain7,9379,9379.000000,1480,1480.000000,32,32.000000,220,220.0,1
749,youtubemusic,12419,4139.666667,2296,765.333333,86,28.666667,522,174.0,3
750,yrf,60384,30192.000000,7682,3841.000000,358,179.000000,2172,1086.0,2
751,zaisha_SRK,7501,7501.000000,1210,1210.000000,6,6.000000,119,119.0,1


In [8]:
print(tweets_df['username'].drop_duplicates())

0           BarackObama
17      ObamaFoundation
18                POTUS
19              bsw5020
25          CivicNation
             ...       
4862      premierleague
4875            Arsenal
4877            ManCity
4886     afcbournemouth
4894      SpursOfficial
Name: username, Length: 753, dtype: object
